# Random Projections

### Rishi Advani, Madison Crim, Sean O'Hagan

#### July 10, 2020

# Eigenfaces

As an application of our work on matrix decomposition and random projection, we decided to use the deterministic and randomized variants of the SVD on a large, real-life dataset. We chose to use the [Labeled Faces in the Wild (LFW) dataset](http://vis-www.cs.umass.edu/lfw/), which consists of ~$8000$ RGB images of dimensions $250\times 250$. It's computationally infeasible for us to use the whole dataset, so we load a subset (1000 images).

### Examples

Let's look at some of the faces in the dataset.

![original faces](images/2020-07-10/original_image_grid.png)

We can flatten each image to represent it as vector of length $250\cdot 250 \cdot 3 = 187500$. This yields a data matrix $A$ of size $(187500,1000)$, which we may now manipulate. After computing the singular value decomposition,
$$A = U \Sigma V^* \,,$$
the columns of $U$ represent an orthonormal basis for the span of the original data, ordered in accordance to their corresponding singular values from greatest to least. Thus, we may truncate $U$ and use the first $k$ columns as a basis for a rank $k$ approximation.

![deterministic eigenfaces](images/2020-07-10/det_eigenfaces_grid.png)

Similarly to deterministic SVD, we may utilize randomized SVD for this task as well. As a refresher, given a desired rank $k$ and an oversampling parameter $p$, we compute a randomized SVD in the following way:

- Define $l=k+p$.
- Let $\Omega \in \mathbb{R}^{n\times l}$, with $\Omega_{ij} \sim N(0,1)$
- Define $Y = A \Omega$
- Compute an orthonormal basis for the columns of $Y$ (i.e. with $QR$ decomposition)
- Define $B = Q^* A$. Note $B$ has dimension $l\times n$
- Compute the SVD of B, $B=\tilde{U}\Sigma V^*$, and let $U = Q\tilde{U}$, yielding $A\approx U\Sigma V^*$

Below are the columns of $U$ obtained using randomized SVD:

![randomized eigenfaces](images/2020-07-10/rand_eigenfaces_grid.png)

~~~

Now that we have a basis, we may represent arbitrary vectors in our image space as linear combinations of basis vectors. To do this, we solve the linear system
$$A\vec{x}=\vec{b}-m \,,$$
where $A$ denotes the matrix of which our basis vectors are columns, $\vec{b}$ denotes our specific image, and $m$ denotes the (column) mean of the dataset. Since we know our basis is orthonormal, we can use a shortcut, computing the coefficients $\vec{x}$ as 
$$\vec{x} = (\vec{b}-m)A \,.$$

Below are examples of approximating images at a certain rank after a change of basis. The reason why this technique is useful is that any arbitrary image thought to be similar to those in the dataset may be represented fairly well using this basis.

![reconstructed image grid](images/2020-07-10/reconstructed_image_grid.png)

![reconstructed sean grid](images/2020-07-10/reconstructed_sean_grid.png)

Please enjoy the follow animations:

Aaron Eckhart | Sean | President Obama | Ellie Goulding
- | - | - | -
![Guy](../examples/eigenfaces/guy250.png) | ![Sean](../examples/eigenfaces/sean250.png) | ![Pres](../examples/eigenfaces/president_obama250.png) | ![Ellie](../examples/eigenfaces/ellie250.png)
![Anim](images/2020-07-10/anim/guy.gif) | ![Anim](images/2020-07-10/anim/sean.gif) | ![Anim](images/2020-07-10/anim/pres.gif) | ![Anim](images/2020-07-10/anim/ellie.gif)

### How to get better results?

- Use more data (and more diverse data)
- Train on data that is representative of the real data that the algorithm will be used on
- Standardize the position and scale of the face within the image

~~~

We now perform an analysis on the cost and accuracy of these methods, as the rank $k$ of the approximation varies. We see an inverse relationship between these quantities, as expected.

![performance graph 1](images/2020-07-10/performance_relative_to_det.png)

Finally, since the randomized SVD is a random process, we may sample it and obtain statistical information. Again, due to concentration of measure, note the extremely low standard deviation in the error.

![performance graph 2](images/2020-07-10/performance_relative_to_orig.png)

~~~

# Least-Squares Approximation

In practice, when trying to solve a linear system of equations $Ax=b$, there may not be an exact solution. One way to find an 'almost solution' is to find the vector $x'$ such that $\lVert Ax'-b \rVert_2$ is minimized, where $A$ is a full rank $m \times n$ matrix such that $m\geq n$. This least squares solution which works by minimizing the sum of the squares of the differences between the entries of $Ax'$ and $b$ and thus solving the solution $Ax=b$ as closely as possible.

If the system has an exact solution, $x'$ is precisely the value of $x$ that solves the system. The reasoning is as follows:

The system is solvable, so there is a vector $x$, such that $Ax=b$. Equivalently, there is a vector $x$ such that $Ax-b = 0$. Then, we have $\lVert Ax-b \rVert_2 = 0$. The L2 norm is nonnegative for all its inputs by definition, so the smallest value it can take is $0$. Thus, $x$ is the vector that minimizes the expression.

We generate a random system and then find the least-squares approximation using a deterministic QR-based method and a randomized method.

## Deterministic QR-based method

We run the algorithm on random matrices of varying dimensions, and plot the time it took to run the algorithm as a function of the input size:

![graph](images/2020-07-10/ls_det.png)

## Randomized method
To solve the least squares problem, we also tried devising a randomized method. Given an integer $k$, this method samples $k$ Gaussian vectors $x$ and keeps the vector that best minimizes $\lVert Ax-b \rVert_2$. We run the algorithm on many random $A$ matrices and $b$ vectors with entries from a standard normal distribution. Unfortunately, this naive algorithm was unable to beat the deterministic one, but we hope to optimize it enough for it to be more efficient eventually. One strategy we are considering is finding more structured ways to randomly sample the $x$ vectors, instead of choosing completely arbitrarily from a standard distribution.

![graph](images/2020-07-10/ls_rand.png)

## Another Idea
Use a random matrix approximation, such as the ID, and then find a least squares solution for this approximation. Testing this with the ID gives better efficency than the above randomized approach. However, it still fails to beat our current determinsitic approach.  

~~~

# What's Next?

We are currently interested in randomizing the kernel mapping in kernel PCA. While we do not have time to go into detail in this presentation, here is a quick demonstration:

Given the following data:

![Data](../examples/kpca/fakedata.png)

We want to find an embedding in which the two groups are linearly separable (and preferably well clustered). Regular PCA (linear kernel) will fail, as the polar symmetry in the data tells us that there is no direction (linear combination of the x and y axes) that accounts for significantly more variance in the data. In order to find meaningful principle components, we utilize kernel PCA, first mapping the data points into a higher dimensional space using a kernel function. Here are some embeddings of the data using various kernels and default parameters:

![Embeds](../examples/kpca/kpca_embeds.png)

As we can see, none of them are linearly separable. Often, we can find a better embedding by playing around with one of the parameters. Here is an animation of embeddings using the RBF/Gaussian kernel as the gamma parameter changes, $0<\gamma<26$.

![Anim](../examples/kpca/anim.gif)

While some values of gamma produce good nice/separable embeddings, choosing a value without guess and check is difficult. We are now investigating a randomized method for determining this kernel mapping, which produces nice output without having to arduously check parameters:

![RKPCA](../examples/kpca/rkpca.png)

# Thank you for listening!

### Questions?